<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/falcon-7b%20and%201b%20comparison/falcon_1b_pubmed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook finetune Falcon-rw-1b on the pubmed_qa dataset https://huggingface.co/datasets/pubmed_qa in order to record its performance and result.

This will then be used to compare with the perfomance and result of the Falcon-7b-sharded-bf16 model with the same dataset.